# Plot groups' analysis
## Set parameters

In [ ]:
# ####################################### GENERAL OPTIONS #######################################
data_root = "./data/experiments/"
plots_root = "./plots/"
branches_to_exclude = ["retina", "VS", "grv", "fiber tracts", "CB"]
normalization = "RelativeDensity"                   # call get_normalization_methods() on a AnimalGroup object to know its available normalization methods
min_animals_cross_correlation = 10                   # 'None' means that ALL the animals must have the region, otherwise the correlation is NaN
plot_regions_with_insufficient_data = True          # what to do with brain regions with less animals than min_animals_cross_correlation
saved_plot_extension = ".html"                      # '.html' for interactive plot
                                                    # '.svg' for vectorized image
                                                    # '.png'/'.jpg'/... for rasterized image


# ########################################## BAR PLOT ###########################################
bar_height = 10_000
bar_use_acronyms = False
bar_save_plot = True
bar_show_plot = False

# ###################################### CORRELATION MATRIX #####################################
matrix_cell_height = 5
matrix_save_plot = True
matrix_show_plot = False

# ######################################## CHORD DIAGRAM ########################################
chord_p_cutoff = 0.05
chord_r_cutoff = 0.9
chord_plot_size = 1200
chord_no_background = False
chord_use_colorscale_edges = True
chord_save_plot = True
chord_show_plot = True
chord_bottom_annotations = dict(
    annotation1 = "This is the first annotation",
    annotation2 = "This is the second annotation",
    annotation3 = "This is the third annotation",
    # howmany annotations desired with the following format:
    # annotations<k> = "<annotation>"
)

In [ ]:
# SOUMNYA FEMALES+MALES - 2 Groups {Stress|Control}
experiment = "soumnya"
groups = [
    "Control",
    "Stress"
]
plots_output_folder = "C-S"

In [ ]:
# SOUMNYA ALL - 2 Groups {Stress|Control} + 2 Groups {Males|Females}
experiment = "soumnya"
groups = [
    "Control (Females)",
    "Stress (Females)",
    "Control (Males)",
    "Stress (Males)"
]
plots_output_folder = "CF-SF-CM-SM"

In [ ]:
# SOUMNYA FEMALES - 2 Groups {Stress|Control}
experiment = "soumnya"
groups = [
    "Control (Females)",
    "Stress (Females)",
]
plots_output_folder = "CF-SF"

In [ ]:
# SOUMNYA MALES - 2 Groups {Stress|Control}
experiment = "soumnya"
groups = [
    "Control (Males)",
    "Stress (Males)"
]
plots_output_folder = "CM-SM"

In [ ]:
# SHILA - 3 Groups {Control|Stress|Resilient}
experiment = "shila"
groups = [
    "Control",
    "Stress",
    "Resilient"
]
plots_output_folder = "C-S-R"

In [ ]:
# SHILA - 2 Groups {Control|Stress+Resilient}
experiment = "shila"
groups = [
    "Control",
    "Stress"
]
plots_output_folder = "C-S"

In [ ]:
# TEST DATA - Sample data
experiment = "test"
groups = [
    "Control",
    "Stress",
    "Resilient"
]
plots_output_folder = "C-S-R"

## Script's code
run all cell below

In [ ]:
import BraiAn
import os

data_input_path = os.path.join(data_root, experiment, "BraiAn_output")
plots_output_path = os.path.join(plots_root, experiment, plots_output_folder)

if not(os.path.exists(plots_output_path)):
    os.makedirs(plots_output_path, exist_ok=True)

In [ ]:
animal_groups = []
for group in groups:
    animal_group = BraiAn.AnimalGroup.from_csv(group, data_input_path, f"cell_counts_{group}.csv")
    animal_groups.append(animal_group)
    print(f"Group '{animal_group.name}' - #animals: {animal_group.n}, marker: {animal_group.marker}")


In [ ]:
# from https://help.brain-map.org/display/api/Downloading+an+Ontology%27s+Structure+Graph
# StructureGraph id=1
path_to_allen_json = "./data/AllenMouseBrainOntology.json"

AllenBrain = BraiAn.AllenBrainHierarchy(path_to_allen_json, branches_to_exclude)
AllenBrain.select_from_csv("./data/AllenSummaryStructures.csv")

In [ ]:
# low_threshold = 700 # Only plot bars with value larger than threshold
# top_threshold = 4_000 # np.inf
# regions_to_plot = BraiAn.regions_to_plot(groups=animal_groups, normalization=normalization, low_threshold=low_threshold, top_threshold=top_threshold)
regions_to_plot = AllenBrain.get_selected_regions() # selects the Summary Strucutures

In [ ]:
fig = BraiAn.plot_groups(normalization, AllenBrain, *animal_groups,
                            selected_regions=regions_to_plot, use_acronyms=bar_use_acronyms, height=bar_height)

if bar_save_plot:
    plot_filename = f"barplot_{plots_output_folder}_{normalization}_{animal_groups[0].marker}{saved_plot_extension}".lower()
    plot_filepath = os.path.join(plots_output_path, plot_filename)
    match saved_plot_extension.lower():
        case ".html":
            fig.write_html(plot_filepath)
        case _:
            fig.write_image(plot_filepath)
if bar_show_plot:
    fig.show()

In [ ]:
groups_cross_correlations = []
for group in animal_groups:
    r, p = group.cross_correlation(normalization, regions_to_plot, min_animals=min_animals_cross_correlation)
    if not plot_regions_with_insufficient_data:
            r.dropna(axis=0, how="all", inplace=True)
            r.dropna(axis=1, how="all", inplace=True)
            p.dropna(axis=0, how="all", inplace=True)
            p.dropna(axis=1, how="all", inplace=True)
    groups_cross_correlations.append((r, p))

In [ ]:
if matrix_save_plot or matrix_show_plot:
    for group, (r,p) in zip(animal_groups, groups_cross_correlations):
        fig = BraiAn.plot_cross_correlation(r, p, cell_height=matrix_cell_height)
        if matrix_save_plot:
            plot_filename = f"correlation_matrix_{group.name}_{normalization}_{group.marker}{saved_plot_extension}".lower()
            plot_filepath = os.path.join(plots_output_path, plot_filename)
            match saved_plot_extension.lower():
                case ".html":
                    fig.write_html(plot_filepath)
                case _:
                    fig.write_image(plot_filepath)
        if matrix_show_plot:
            fig.show()

In [ ]:
for group, (r,p) in zip(animal_groups, groups_cross_correlations):
    fig = BraiAn.draw_chord_plot(r, p, chord_r_cutoff, chord_p_cutoff, AllenBrain,
                                ideograms_a=50,
                                title=f"{group.name} connectomics graph from Pearson correlation (|r| >= {chord_r_cutoff}, p >= {chord_p_cutoff})",
                                size=chord_plot_size,
                                no_background=chord_no_background,
                                colorscale_edges=chord_use_colorscale_edges,
                                **chord_bottom_annotations
    )
    if chord_save_plot:
        plot_filename = f"chord_plot_{group.name}_{normalization}_{group.marker}{saved_plot_extension}".lower()
        plot_filepath = os.path.join(plots_output_path, plot_filename)
        match saved_plot_extension.lower():
            case ".html":
                fig.write_html(plot_filepath)
            case _:
                fig.write_image(plot_filepath)
    if chord_show_plot:
        fig.show()